<a href="https://colab.research.google.com/github/ddib247/deez247/blob/main/ContextOS_Manager_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# %% [markdown]
# # ContextOS Manager // Release Candidate 1
# **Role:** The Manager (Data Processing & Analysis)
# **Objective:** Process "DreamerDump" inputs, perform NLP clustering, and update the "TaskBacklog".
# **Version:** v0.3.1 (JSON-Aware & Schema Aligned)

# %% [code]
# ==========================================
# CELL 1: SETUP & CONFIGURATION
# ==========================================
import os
import pandas as pd
import nltk
import json
from google.colab import drive

# --- USER CONFIGURATION (Simple Variables) ---
# @title Colab Configuration
# Where does your ContextOS project file folder live in Drive?
DRIVE_FOLDER = "/content/drive/MyDrive/Area 1225/Project 1" # @param {type:"string"}
DB_FILENAME = "ContextOS_Master_DB" # @param {type:"string"}

# Constants for File Paths
BASE_PATH = f"/content/drive/My Drive/{DRIVE_FOLDER}"
DREAMER_DUMP_PATH = f"{BASE_PATH}/DreamerDump.csv"
BACKLOG_PATH = f"{BASE_PATH}/TaskBacklog.csv"

# Mount Google Drive
print("🔌 Connecting to Google Drive...")
drive.mount('/content/drive')

# Verify Directory
if not os.path.exists(BASE_PATH):
    print(f"⚠️ Warning: Folder '{BASE_PATH}' not found.")
    print(f"   Creating '{BASE_PATH}' now...")
    os.makedirs(BASE_PATH, exist_ok=True)
else:
    print(f"✅ ContextOS Folder Found: {BASE_PATH}")

# Download NLTK resources (Run once)
print("📚 Loading NLP Libraries...")
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter

print("🚀 System Ready.")

# %% [code]
# ==========================================
# CELL 2: THE MANAGER INTELLIGENCE (FUNCTIONS)
# ==========================================

def load_data():
    """Loads the CSV databases. Creates them if missing."""
    data = {}

    # Load DreamerDump
    # Schema matches 'context_bridge.gs':
    # [Dream_ID, Timestamp, Source_Type, Core_Intent, JSON_Payload, Status]
    if os.path.exists(DREAMER_DUMP_PATH):
        try:
            data['dreams'] = pd.read_csv(DREAMER_DUMP_PATH)
            print(f"📖 Loaded {len(data['dreams'])} dreams.")
        except Exception as e:
            print(f"⚠️ Error loading DreamerDump: {e}")
            data['dreams'] = pd.DataFrame(columns=["Dream_ID", "Timestamp", "Source_Type", "Core_Intent", "JSON_Payload", "Status"])
    else:
        print("✨ No DreamerDump found. Creating new dataframe.")
        data['dreams'] = pd.DataFrame(columns=["Dream_ID", "Timestamp", "Source_Type", "Core_Intent", "JSON_Payload", "Status"])

    # Load Backlog
    if os.path.exists(BACKLOG_PATH):
        try:
            data['backlog'] = pd.read_csv(BACKLOG_PATH)
            print(f"📋 Loaded {len(data['backlog'])} backlog tasks.")
        except Exception as e:
             print(f"⚠️ Error loading Backlog: {e}")
             data['backlog'] = pd.DataFrame(columns=["ID", "Task Name", "Context_Tag", "Social_Req", "Opportunity_Score", "Visual_Reward", "Est_Duration", "Deadline", "Dynamic_Score", "Status"])
    else:
        print("✨ No TaskBacklog found. Creating new dataframe.")
        # Schema matches v0.3.1 Backlog
        data['backlog'] = pd.DataFrame(columns=["ID", "Task Name", "Context_Tag", "Social_Req", "Opportunity_Score", "Visual_Reward", "Est_Duration", "Deadline", "Dynamic_Score", "Status"])

    return data

def extract_raw_text(row):
    """Extracts raw text stream from JSON payload for NLP analysis."""
    try:
        payload = json.loads(row['JSON_Payload'])
        return payload.get('content', {}).get('raw_stream', row['Core_Intent'])
    except:
        return str(row['Core_Intent'])

def analyze_rabbit_holes(df_dreams):
    """
    NLP Logic: Clusters keywords from the raw stream inside the JSON.
    """
    if df_dreams.empty:
        return {}

    # Extract text from JSON
    text_blob = " ".join(df_dreams.apply(extract_raw_text, axis=1).tolist())

    # Tokenize & Clean
    tokens = word_tokenize(text_blob.lower())
    stop_words = set(stopwords.words('english'))
    clean_tokens = [word for word in tokens if word.isalnum() and word not in stop_words]

    # Count Frequencies
    freq = Counter(clean_tokens)
    return freq.most_common(10)

def promote_to_task(dream_row, backlog_df):
    """
    Converts a Dream Row into a specific First Draft Task using JSON data.
    """
    new_id = len(backlog_df) + 1

    # Default values (Fallbacks)
    task_name = f"Draft: {dream_row['Core_Intent']}"
    context_tag = "Home"
    est_duration = 15
    visual_reward = 5

    # Try to parse the JSON for smarter defaults
    try:
        payload = json.loads(dream_row['JSON_Payload'])

        # 1. Extract the "First Draft" Task
        if 'first_draft_tasks' in payload and len(payload['first_draft_tasks']) > 0:
            draft = payload['first_draft_tasks'][0]
            task_name = draft.get('task_name', task_name)
            context_tag = draft.get('context_tag', context_tag)
            est_duration = draft.get('est_duration', est_duration)

        # 2. Extract Visual Reward Potential
        if 'analysis' in payload:
            visual_reward = payload['analysis'].get('visual_reward_potential', visual_reward)

    except Exception as e:
        print(f"   ⚠️ JSON Parse Error for Dream {dream_row.get('Dream_ID', '?')}: {e}")

    # Construct the Row
    new_task = {
        "ID": f"{new_id:03d}",
        "Task Name": task_name,
        "Context_Tag": context_tag,
        "Social_Req": "Solo", # Default to Solo for focused work
        "Opportunity_Score": 5,
        "Visual_Reward": visual_reward,
        "Est_Duration": est_duration,
        "Deadline": "", # Manager adds this manually later
        "Dynamic_Score": 0,
        "Status": "Pending"
    }
    return new_task

print("🧠 Manager Logic Loaded (v0.3.1).")

# %% [code]
# ==========================================
# CELL 3: EXECUTION (RUN THE MANAGER)
# ==========================================

# 1. Load the Brain
db = load_data()

# 2. Analyze Dreams (The "Review" Phase)
if not db['dreams'].empty:
    print("\n🔍 Analyzing Rabbit Holes (Top Keywords):")
    themes = analyze_rabbit_holes(db['dreams'])
    for word, count in themes:
        print(f"   - {word}: {count} occurrences")

# 3. Auto-Promotion
unprocessed = db['dreams'][db['dreams']['Status'] == "Unprocessed"]

if not unprocessed.empty:
    print(f"\n⚙️ Processing {len(unprocessed)} new dreams...")
    new_tasks = []

    for index, row in unprocessed.iterrows():
        # Create Task
        task = promote_to_task(row, db['backlog'])
        new_tasks.append(task)

        # Mark Dream as Processed
        db['dreams'].at[index, 'Status'] = "Processed"
        print(f"   -> Promoted: {task['Task Name']} (Ctx: {task['Context_Tag']})")

    # Append to Backlog
    new_task_df = pd.DataFrame(new_tasks)
    db['backlog'] = pd.concat([db['backlog'], new_task_df], ignore_index=True)

    # Save Changes to Drive
    # NOTE: Uncomment these lines to actually write to your Drive files
    # db['dreams'].to_csv(DREAMER_DUMP_PATH, index=False)
    # db['backlog'].to_csv(BACKLOG_PATH, index=False)
    print("💾 Changes simulated. Uncomment save lines to write to Drive.")

else:
    print("\n✅ All dreams are processed.")

# %% [code]
# ==========================================
# CELL 4: OPTIONAL SELF-REVIEW (DEBUGGER)
# ==========================================
# Run this cell to check data integrity or logic errors

def diagnostic_review(data):
    print("--- 🩺 DATA DIAGNOSTICS ---")

    # Check Backlog Health
    bl = data['backlog']
    print(f"Task Backlog: {len(bl)} rows")

    if 'Dynamic_Score' in bl.columns:
        null_scores = bl['Dynamic_Score'].isnull().sum()
        if null_scores > 0:
            print(f"⚠️ ALERT: {null_scores} tasks have no Dynamic Score.")

    # Check Context Tags
    if 'Context_Tag' in bl.columns:
        tags = bl['Context_Tag'].unique()
        print(f"Active Context Tags: {tags}")

    # Check Dream Parse
    if not data['dreams'].empty:
        sample_json = data['dreams'].iloc[0].get('JSON_Payload')
        print(f"Sample Dream JSON Preview: {str(sample_json)[:50]}...")

    print("---------------------------")

# Run Diagnostic
diagnostic_review(db)

🔌 Connecting to Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ ContextOS Folder Found: /content/drive/My Drive//content/drive/MyDrive/Area 1225/Project 1
📚 Loading NLP Libraries...
🚀 System Ready.
🧠 Manager Logic Loaded (v0.3.1).
✨ No DreamerDump found. Creating new dataframe.
✨ No TaskBacklog found. Creating new dataframe.

✅ All dreams are processed.
--- 🩺 DATA DIAGNOSTICS ---
Task Backlog: 0 rows
Active Context Tags: []
---------------------------


### Adding a Sample Dream to `DreamerDump`

This cell demonstrates how to manually add a new 'dream' entry to the `db['dreams']` DataFrame. Each dream should ideally have a unique `Dream_ID`, `Timestamp`, `Source_Type`, `Core_Intent`, a `JSON_Payload` (which is a JSON string), and a `Status` (initially 'Unprocessed').

In [8]:
import uuid
import datetime
import json

# Define a new sample dream
new_dream_data = {
    "Dream_ID": str(uuid.uuid4()), # Generate a unique ID
    "Timestamp": datetime.datetime.now().isoformat(),
    "Source_Type": "Manual_Input",
    "Core_Intent": "Explore new project idea: AI-powered task prioritization",
    "JSON_Payload": json.dumps({
        "content": {
            "raw_stream": "I was thinking about how to make task management more efficient. What if an AI could look at my notes, emails, and calendar to suggest the most important tasks? It could identify dependencies and potential blockers. This needs to be explored. Maybe a visual dashboard would help."
        },
        "first_draft_tasks": [
            {
                "task_name": "Research AI task prioritization algorithms",
                "context_tag": "Research",
                "est_duration": 60
            }
        ],
        "analysis": {
            "visual_reward_potential": 8
        }
    }),
    "Status": "Unprocessed"
}

# Convert the new dream into a DataFrame row
new_dream_df = pd.DataFrame([new_dream_data])

# Append to the existing dreams DataFrame
db['dreams'] = pd.concat([db['dreams'], new_dream_df], ignore_index=True)

print("New dream added to db['dreams']:")
display(db['dreams'].tail())

# IMPORTANT: To make this change persistent, you must uncomment and run the following line in Cell 3:
# db['dreams'].to_csv(DREAMER_DUMP_PATH, index=False)


New dream added to db['dreams']:


,Dream_ID,Timestamp,Source_Type,Core_Intent,JSON_Payload,Status
0,21863f85-a5a6-4576-b176-a2a5ae9ce45e,2025-12-24T09:51:59.784023,Manual_Input,Explore new project idea: AI-powered task prio...,"{""content"": {""raw_stream"": ""I was thinking abo...",Unprocessed
